In [825]:
"""
Name: Kimberly Nestor
Date: 06/16/2021
Class: CS 767
Description: This program is a genetic algorithm that solves the travelling salesman problem by finding the shortest path between cities.
"""

'\nName: Kimberly Nestor\nDate: 06/16/2021\nClass: CS 767\nDescription: This program is a genetic algorithm that solves the travelling salesman problem by finding the shortest path between cities.\n'

In [826]:
import numpy as np
import pandas as pd
import random

import sys

from google.colab import drive
drive.mount('/content/drive')
dpath = '/content/drive/MyDrive/BU/CLASSES/2021sum/CS767/Homework/Module5/code/route_info.rtf'
dpath_new = '/content/drive/MyDrive/BU/CLASSES/2021sum/CS767/Homework/Module5/code/route_info_new.rtf'



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [827]:
#lambda functions
all_isdigit = lambda list: all([i.replace('.','',1).isdigit() for i in list])
not_zero = lambda list: [i for i in list if float(i[1]) != 0]
sort_node = lambda x: sorted(not_zero(x), key=lambda x:x[1])

def gen_pop(pop_size):
  "This function returns a list of randomly generated genes to use as \
  routes in the TSP."
  pop_lst = []

  for i in range(pop_size):
    gene = [0] + list(np.random.randint(1, 3, 3)) + [0]
    pop_lst.append(gene)

  return(pop_lst)


def crossover(parent1, parent2):
  "This function takes two parent as arrays and produces a children using the \
  single point crossover method."
  co_pt = np.random.randint(1,len(parent1)-1)
  child = parent1[:co_pt] + parent2[co_pt:]
  
  return(child)
  # child1 = parent1[:co_pt] + parent2[co_pt:]
  # child2 = parent2[:co_pt] + parent1[co_pt:]
  # return(child1, child2)


def mutation(child):
  "This function takes a child as input and performs a mutation on one value \
  of the numerical representation. A while loop is used to ensure a mutation \
  has been performed on the returned child."
  mu_loc, mu_num = np.random.randint(1,len(child)-1), \
                   np.random.randint(1,len(child)-2)
  while child[mu_loc]:
    if child[mu_loc] != mu_num:
      child[mu_loc] = mu_num
      return(child)
    else:
      mu_num = np.random.randint(1,len(child)-2)


def fitness(gene, graph):
  "This function takes a gene as input, checks the graph for distances of nodes \
  and returns fitness. Lower fitness is optimal."
  nodes_lst = [0]
  dist = 0
  gene = gene[1:]
  # curr = 0
  for i in range(len(graph)):
    connec_node = iter( [i for i in sort_node(graph[ nodes_lst[-1] ]) if i[0] !=0] )
    connec_node2 = iter( [i for i in sort_node(graph[ nodes_lst[-1] ]) if i[0] !=0][1:] )

    if gene[i] == 1:
      curr = next(connec_node)
      while curr != 0:
        if curr[0] not in nodes_lst:
          dist += curr[1]
          nodes_lst.append(curr[0])
          break
        else:
          try:
            curr = next(connec_node)
          except(StopIteration):
            curr = set(nodes_lst).symmetric_difference(set(range(len(graph))))
            curr = graph[nodes_lst[-1]][list(curr)[0]]
            
    elif gene[i] == 2:
        curr = next(connec_node2)
        while curr != 0:
          if curr[0] not in nodes_lst:
            dist += curr[1]
            nodes_lst.append(curr[0])
            break
          else:
            try:
              curr = next(connec_node2)
            except(StopIteration):
              curr = set(nodes_lst).symmetric_difference(set(range(len(graph))))
              curr = graph[nodes_lst[-1]][list(curr)[0]]
              
    elif gene[i] == 0:
      home_dist = graph[nodes_lst[-1]] [0] [1]
      dist += home_dist

  return(dist)


def tournament(parent1, parent2, child, graph):
  "This function determines the fittest out of the parents and the child."
  tourn_lst = [parent1, parent2, child]
  fit = enumerate([fitness(i, graph) for i in tourn_lst])
  winners = [tourn_lst[idx] for idx in [i[0] for i in sorted(fit, \
                                          key=lambda x:x[1])[:2]]]

  return(winners)


def gen_alg(pop_size, num_gener, mut_prob, route_data):
  "This function is a genetic algorithm that uses crossover and mutation to \
  develop new popultations over generations with the goal being to find the \
  shorest route in the TSP. Crossover function uses probability of shortest \
  distance to pick random parents for crossover. Tournament is used to fill new \
  pop over generations, where out of P1, P2 and child only the two fittest move \
  on to the new generation. "
  #init pop,
  pop = gen_pop(pop_size)
  new_pop = []

  #run genetic algorithm for num_gener
  for gen in range(num_gener):
    if gen == 0:
      no_mut_lst = []
      mut_lst = []

      #find fitness, pick parents from pop based on fitness weight
      fit = [fitness(i, route_data) for i in pop]
      fit_prob = 1/np.array(fit)/sum(np.unique(fit)) #inverse
      
      #generate new pop based on fitness, use tournament
      while len(no_mut_lst) < pop_size:
        rand_parents = random.choices(pop, weights=fit_prob, k=2)
        new_child = crossover(rand_parents[0], rand_parents[1])

        fittest = tournament(rand_parents[0], rand_parents[1], new_child, route_data)
        no_mut_lst = no_mut_lst + fittest

      #perform mutation on select prob of pop
      for child in no_mut_lst:
        if random.random() < mut_prob:
          mut_child = mutation(child)
          mut_lst.append(mut_child)
        else:
          mut_lst.append(child)
      
      #fittest of generation
      fittest = sorted(list(enumerate([fitness(i, route_data) for i in mut_lst])), \
                       key=lambda x:x[1])[0]
      fittest = mut_lst[fittest[0]], fittest[1]
      
      #becomes new generation
      print(f"Generation: {gen}.  Route: {fittest[0]}.  Distance: {fittest[1]}")
      new_pop = mut_lst
    
    else:
      no_mut_lst = []
      mut_lst = []

      #find fitness, pick parents from pop based on fitness weight
      fit = [fitness(i, route_data) for i in new_pop]
      fit_prob = 1/np.array(fit)/sum(np.unique(fit)) #inverse
      
      #generate new pop based on fitness, use tournament
      while len(no_mut_lst) < pop_size:
        rand_parents = random.choices(new_pop, weights=fit_prob, k=2)
        new_child = crossover(rand_parents[0], rand_parents[1])

        fittest = tournament(rand_parents[0], rand_parents[1], new_child, route_data)
        no_mut_lst = no_mut_lst + fittest

      #perform mutation on select prob of pop
      for child in no_mut_lst:
        if random.random() < mut_prob:
          mut_child = mutation(child)
          mut_lst.append(mut_child)
        else:
          mut_lst.append(child)
      
      #fittest of generation
      fittest = sorted(list(enumerate([fitness(i, route_data) for i in mut_lst])), \
                       key=lambda x:x[1])[0]
      fittest = mut_lst[fittest[0]], fittest[1]
      
      #becomes new generation
      print(f"Generation: {gen}.  Route: {fittest[0]}.  Distance: {fittest[1]}")
      new_pop = mut_lst


# parent1, parent2 = [0, 2, 2, 1, 0], [0, 1, 1, 1, 0]
# cross_child = crossover(parent1, parent2)
# mut_child = mutation(cross_child)

# print("cross_child = ", cross_child)
# print("mut_child = ", mut_child)



In [828]:
#import given data
data = np.squeeze(pd.read_csv(dpath, sep='delimiter', engine='python').values)

routes_lst = [i.split(',') for i in [i.replace('}', '') for i in \
                                  [i.replace('\\', '') for i in data]]]
routes_lst = list(filter(all_isdigit, routes_lst))
routes_lst = [list(map(float, i)) for i in routes_lst] #B,S,L,M

enum_routes = [list(enumerate(i)) for i in routes_lst]

#convert to dict
routes_dict = {i: enum_routes[i] for i in range(len(enum_routes))}

#run genetic algorithm to find shortest route
gen_alg(pop_size=25, num_gener=5, mut_prob=0.20, route_data = routes_dict)

Generation: 0.  Route: [0, 1, 1, 1, 0].  Distance: 18.4
Generation: 1.  Route: [0, 1, 1, 2, 0].  Distance: 18.4
Generation: 2.  Route: [0, 1, 1, 2, 0].  Distance: 18.4
Generation: 3.  Route: [0, 1, 1, 1, 0].  Distance: 18.4
Generation: 4.  Route: [0, 1, 1, 2, 0].  Distance: 18.4


In [829]:
#import new data
data = np.squeeze(pd.read_csv(dpath_new, sep='delimiter', engine='python').values)

routes_lst = [i.split(',') for i in [i.replace('}', '') for i in \
                                  [i.replace('\\', '') for i in data]]]
routes_lst = list(filter(all_isdigit, routes_lst))
routes_lst = [list(map(float, i)) for i in routes_lst] #B,S,L,M

enum_routes = [list(enumerate(i)) for i in routes_lst]

#convert to dict
routes_dict = {i: enum_routes[i] for i in range(len(enum_routes))}

#run genetic algorithm to find shortest route
gen_alg(pop_size=25, num_gener=5, mut_prob=0.20, route_data = routes_dict)

Generation: 0.  Route: [0, 1, 2, 1, 0].  Distance: 20.799999999999997
Generation: 1.  Route: [0, 1, 2, 1, 0].  Distance: 20.799999999999997
Generation: 2.  Route: [0, 1, 2, 1, 0].  Distance: 20.799999999999997
Generation: 3.  Route: [0, 1, 2, 2, 0].  Distance: 20.799999999999997
Generation: 4.  Route: [0, 1, 2, 1, 0].  Distance: 20.799999999999997
